## Data Preparation

In [11]:
import pandas as pd

In [12]:
#make path relative
df = pd.read_csv("PythonProjects/Project_1/population-growth-the-annual-change-of-the-population.csv")

In [13]:
df

,Entity,Code,Year,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Population change (thousands)","Medium fertility variant, 2020 - 2099: Annually interpolated demographic indicators - Population change (thousands)"
0,Afghanistan,AFG,1950,87538.0,NaN
1,Afghanistan,AFG,1951,91918.0,NaN
2,Afghanistan,AFG,1952,99744.0,NaN
3,Afghanistan,AFG,1953,107636.0,NaN
4,Afghanistan,AFG,1954,115628.0,NaN
...,...,...,...,...,...
36895,Zimbabwe,ZWE,2095,NaN,17903.0
36896,Zimbabwe,ZWE,2096,NaN,13532.0
36897,Zimbabwe,ZWE,2097,NaN,9516.0
36898,Zimbabwe,ZWE,2098,NaN,5922.0


In [14]:
df = df.drop(['Code'], axis = 1)

In [15]:
df2 = df[df['Entity'] == "World"]

In [16]:
df2

,Entity,Year,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Population change (thousands)","Medium fertility variant, 2020 - 2099: Annually interpolated demographic indicators - Population change (thousands)"
36300,World,1950,47162696.0,NaN
36301,World,1951,47220851.0,NaN
36302,World,1952,46789623.0,NaN
36303,World,1953,46992190.0,NaN
36304,World,1954,47703160.0,NaN
...,...,...,...,...
36445,World,2095,NaN,6616506.0
36446,World,2096,NaN,5877417.0
36447,World,2097,NaN,5118115.0
36448,World,2098,NaN,4334758.0


In [17]:
df2 = df2.drop(['Entity'], axis = 1)

In [18]:
df2

,Year,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Population change (thousands)","Medium fertility variant, 2020 - 2099: Annually interpolated demographic indicators - Population change (thousands)"
36300,1950,47162696.0,NaN
36301,1951,47220851.0,NaN
36302,1952,46789623.0,NaN
36303,1953,46992190.0,NaN
36304,1954,47703160.0,NaN
...,...,...,...
36445,2095,NaN,6616506.0
36446,2096,NaN,5877417.0
36447,2097,NaN,5118115.0
36448,2098,NaN,4334758.0


In [19]:
df2 = df2.drop(['Medium fertility variant, 2020 - 2099: Annually interpolated demographic indicators - Population change (thousands)'], axis = 1)

In [20]:
df2

,Year,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Population change (thousands)"
36300,1950,47162696.0
36301,1951,47220851.0
36302,1952,46789623.0
36303,1953,46992190.0
36304,1954,47703160.0
...,...,...
36445,2095,NaN
36446,2096,NaN
36447,2097,NaN
36448,2098,NaN


In [27]:
df2 = df2.set_index("Year")

In [28]:
df2

,"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Population change (thousands)"
Year,
1950,47162696.0
1951,47220851.0
1952,46789623.0
1953,46992190.0
1954,47703160.0
...,...
2095,NaN
2096,NaN
2097,NaN


In [29]:
df2 = df2.rename(columns = {"Estimates, 1950 - 2020: Annually interpolated demographic indicators - Population change (thousands)" : "Population"})

In [30]:
df2

,Population
Year,
1950,47162696.0
1951,47220851.0
1952,46789623.0
1953,46992190.0
1954,47703160.0
...,...
2095,NaN
2096,NaN
2097,NaN


In [31]:
df2 = df2.dropna()

In [32]:
df2

,Population
Year,
1950,47162696.0
1951,47220851.0
1952,46789623.0
1953,46992190.0
1954,47703160.0
...,...
2015,84367770.0
2016,84033390.0
2017,83537542.0
